In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import functions_forcrawl

edit = functions_forcrawl.All_func()

In [ ]:
def clean_semicolon(st):
    
    if re.search(';$|; $', st, flags=re.S):
        return re.sub(';$|; $', '', st, flags=re.S).strip()
    
    else:
        return st
    
def add_minutes(t, m):
    
    hours, minutes = map(int, t.split(":"))
    total_minutes = 60 * hours + minutes + m
    new_hours = total_minutes // 60
    new_minutes = total_minutes % 60
    return "{:02d}:{:02d}".format(new_hours % 24, new_minutes)


In [ ]:
import requests

cookies = {
    'joe-chnlcustid': '0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
    'PHPSESSID': 'lbwww6~880a7f8df3aa697b58175fe27ee98acf',
    'CookieConsent': '{stamp:%27-1%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27implied%27%2Cver:1%2Cutc:1678764599279%2Cregion:%27IN%27}',
}

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'joe-chnlcustid=0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49; PHPSESSID=lbwww6~880a7f8df3aa697b58175fe27ee98acf; CookieConsent={stamp:%27-1%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27implied%27%2Cver:1%2Cutc:1678764599279%2Cregion:%27IN%27}',
    'Origin': 'https://online.eccmid.org',
    'Referer': 'https://online.eccmid.org/programme-live-1?coday=2023-04-16&embed=&typeHideAllBut=23,21,24,4,3,22,1,6,17,15,11,8,9,5,7,12,25,10',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'idCat': '1',
    'sessionRef': 'OS200',
    'timezone': 'Europe/Paris',
    'defaultTimezone': 'Europe/Paris',
    'dispCountry': '',
    'target': '_self',
    'embed': 'false',
    'dispCities': '',
    'firstnameFull': '',
    'page': '',
}

response = requests.post(
    'https://online.eccmid.org/fo-user-display-session-details.php',
    cookies=cookies,
    headers=headers,
    data=data,
)

In [ ]:
dates = ['02-15', '02-16', '03-15', '03-16', '04-14', '04-15', '04-16', '04-17', '04-18', '05-24', '05-25']

In [ ]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [ ]:
se_no = 1
for dat in dates:
    
    session_url = f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&typeHideAllBut=23,21,24,4,3,22,1,6,17,15,11,8,9,5,7,12,25,10'
    reqout = requests.get(session_url, headers=headers, cookies=cookies)
    soup = BS(reqout.content, 'lxml')
    
    outer_row = soup.find_all(class_='row live-program-card-row-grid')
    locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')
    
    for ou in outer_row:

        col_div = 0
        
        for i_id in ou.find_all(class_='border text-center program-grid-calandar'):
            
            if i_id.find(class_='programme-session-card programme-session-display border'):
                
                session_number = f'S{se_no}'
                print(f'=====**====={session_number}=====**=======', col_div)
                i_id = i_id.find(class_='programme-session-card programme-session-display border')

                data['sessionRef'] = i_id.get('data-session-reference')
                if len(str(soup)) < 20:
                    print('%%%%%%%%%%%%% check %%%%%%%%%%%%%')
                    print(i_id.get('data-session-reference'))
                    continue
                
                req = requests.post(
                    'https://eccmid2023.key4.live/fo-user-display-session-details.php',
                    cookies=cookies,
                    headers=headers,
                    data=data,)

                soupin = BS(req.content, 'lxml')
                sess_data = soupin.find(id='modal-session-content')

                # -------------------------------------    date time location ya type     ------------------------------------
                sess_date = ''
                sess_time = ''
                sess_loc = ''
                sess_type = ''
                sess_loc = ''
                sess_loc = locations[col_div].text.strip()
                date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                sess_date = date_time_type.split(',')[0].strip()
                sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                sess_type = date_time_type.split('CET')[-1].strip()

                # --------------------------------------  category or type and title -----------------------------------------------------
                sess_cate = ''
                sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                # --------------------------------------- abstract text OR co organised ---------------------------------------------
                co_org = ''
                sess_text = ''
                if sess_data.find(class_='modal-session-organized-by mt-2'):
                    co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                if sess_data.find(class_='modal-session-desc mt-2'):
                    sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()
                    
                sess_text = co_org + ' ' + sess_text

                # ------------------------------------------- session ka authors aff --------------------------------------------------
                sess_author = ''
                sess_aff = ''

                if sess_data.find(class_='modal-session-moderators'):
                    for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                        aff = i.find(class_='modal-session-moderator-country').text.strip()
                        auth = i.text.strip()
                        auth = auth.replace(aff, '').split('-')[0]
                        aff = aff.replace(',', '').strip()

                        sess_author += auth + '; '
                        sess_aff += aff + '; '

                sess_author = clean_semicolon(sess_author)
                sess_aff = clean_semicolon(sess_aff)
                sess_date = sess_date + '/2023'
                sess_type = re.sub(sess_loc, '', sess_type)
                
                art_sponser = ''
                if soupin.find(class_='modal-session-organized-by mt-2'):
                    spons = soupin.find(class_='modal-session-organized-by mt-2').text.strip()
                    if re.search('^Organized ', spons, flags=re.S):
                        art_sponser = spons
                
                

                print(f'sess date :- {sess_date}')
                print(f'sess time :- {sess_time}')
                print(f'sess loca :- {sess_loc}')
                print(f'sess type :- {sess_type}')
                print(f'sess cate :- {sess_cate}')
                print(f'sess title :- {sess_title}')
                print(f'abs text :- {sess_text}')
                print(f'sess auth :- {sess_author}')
                print(f'sess aff :- {sess_aff}')

                dic['session_id'].append(session_number)
                dic['news_type'].append('Session')
                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(sess_title)
                dic['url'].append(session_url)
                dic['authors'].append(sess_author)
                dic['author_affiliation'].append(sess_aff)
                dic['abstract_text'].append(sess_text)
                dic['date'].append(sess_date)
                dic['start_time'].append(sess_time.split('-')[0].strip())
                dic['end_time'].append(sess_time.split('-')[-1].strip())
                dic['location'].append(sess_loc)
                dic['session_title'].append(sess_title)
                dic['session_type'].append(sess_type)
                dic['category'].append(sess_cate)
                dic['sub_category'].append('')
                dic['disclosure'].append('')
                print('======'*20)
                
                # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*
                
                abs_data = sess_data.find(class_='modal-sessions-interventions')
                if abs_data:
                    
                    pno = 1
                    for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                        abs_title = ab.find(style='font-weight: bold').text.strip()

                        # abs author
                        abs_text = ''
                        abs_author = ''
                        abs_aff = ''
                        if ab.find(class_=re.compile('modal-session-faculties mb-1')):
                            ab_authaff = ab.find(class_=re.compile('modal-session-faculties mb-1')).text.strip()
                            abs_author = ', '.join(ab_authaff.split(',')[:-1])
                            abs_aff = ab_authaff.split(',')[-1]

                        if abs_title == '':
                            abs_title = 'TBD'
                            
                        # source id...... time shit
                        
                        ar_id = ''
                        art_time = ''
                        
                        if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                            src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                            if src_code:

                                if re.search('^\d{1,6}', src_code):
                                    ar_id = src_code + '-' + str(pno)
                                    art_time = ''
                                elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                    str_pno = str(pno)
                                    if len(str_pno) == 1:
                                        str_pno = '0' + str_pno
                                    ar_id = src_code + str_pno
                                    art_time = sess_time
                        else:
                            ar_id = 'QC'
                        
                        abs_text = art_sponser + abs_text
                        
                        print(ar_id, '===', art_time)
                        print(f'abs title :- {abs_title}')
                        print(f'abs auth :- {abs_author}')
                        print(f'abs aff :- {abs_aff}')
                        print(f'abs text :- {abs_text}')

                        dic['session_id'].append(session_number)
                        dic['news_type'].append('Abstract')
                        dic['source_id'].append(ar_id)
                        dic['manual_id'].append('')
                        dic['article_title'].append(abs_title)
                        dic['url'].append(session_url)
                        dic['authors'].append(abs_author)
                        dic['author_affiliation'].append(abs_aff)
                        dic['abstract_text'].append(abs_text)
                        dic['date'].append(sess_date)
                        dic['start_time'].append(sess_time.split('-')[0].strip()) # time nahi diya hai abhi check if updated
                        dic['end_time'].append(sess_time.split('-')[-1].strip()) # for now add session time
                        dic['location'].append(sess_loc)
                        dic['session_title'].append(sess_title)
                        dic['session_type'].append(sess_type)
                        dic['category'].append(sess_cate)
                        dic['sub_category'].append('')
                        dic['disclosure'].append('')
                        print('-*-+-'* 17)
                        pno += 1
                        
                se_no += 1
                print('inr', se_no)
                
            col_div += 1
            

In [ ]:
df = pd.DataFrame(dic)
df

### add 30 mins time with pd

In [ ]:
sestit = set()
c = 0
for i in df['source_id']:
    i = str(i)
    if re.search('^\d{1,10}', i):
        sestit.add(df.iloc[c]['session_title'])
    c += 1

In [ ]:


for tit in sestit:
    c = 0
    grp = []
    for i in df['session_title']:
        if i == tit:
            grp.append(c)
            print(c)
        c += 1
        
    if len(grp) >= 3:

        stime = df.iloc[grp[0]]['start_time']
        entime = df.iloc[grp[0]]['end_time']

        nstim = stime
        netim = add_minutes(stime, 30)

        df.iat[grp[1], df.columns.get_loc('start_time')] = nstim
        df.iat[grp[1], df.columns.get_loc('end_time')] = netim

        for tim in grp[2:-1]:
            print(nstim, netim)

            nstim = add_minutes(nstim, 30)
            netim = add_minutes(netim, 30)

            df.iat[tim, df.columns.get_loc('start_time')] = nstim
            df.iat[tim, df.columns.get_loc('end_time')] = netim

        lst_st = df.iloc[grp[-2]]['end_time']
        lst_en = ''

        if str(add_minutes(df.iloc[grp[-2]]['end_time'], 30)).strip() == str(entime).strip():
            print('same')
            lst_en = entime

        df.iat[grp[-1], df.columns.get_loc('start_time')] = lst_st
        df.iat[grp[-1], df.columns.get_loc('end_time')] = lst_en


    else:
        
        for nc in grp[1:]:
            df.iat[nc, df.columns.get_loc('start_time')] = 'check'
    

In [ ]:
df

### scrape with filter one by one

In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import functions_forcrawl

edit = functions_forcrawl.All_func()

dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

cookies = {
    'joe-chnlcustid': '0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
    'CookieConsent': '{stamp:%27-1%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27implied%27%2Cver:1%2Cutc:1678764599279%2Cregion:%27IN%27}',
    'PHPSESSID': 'lbwww5~545a52322eb94c2d8de9886aecaf76ce',
}

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'joe-chnlcustid=0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49; CookieConsent={stamp:%27-1%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27implied%27%2Cver:1%2Cutc:1678764599279%2Cregion:%27IN%27}; PHPSESSID=lbwww5~545a52322eb94c2d8de9886aecaf76ce',
    'Origin': 'https://online.eccmid.org',
    'Referer': 'https://online.eccmid.org/programme-live-1?coday=2023-02-15&embed=&typeHideAllBut=23,21,24,4,3,22,1,6,17,15,11,8,9,5,7,12,25,10',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'idCat': '1',
    'sessionRef': 'SY001',
    'timezone': 'Europe/Paris',
    'defaultTimezone': 'Europe/Paris',
    'dispCountry': '',
    'target': '_self',
    'embed': 'false',
    'dispCities': '',
    'firstnameFull': '',
    'page': '',
}

response = requests.post(
    'https://online.eccmid.org/fo-user-display-session-details.php',
    cookies=cookies,
    headers=headers,
    data=data,
)

dates = ['02-15', '02-16', '03-15', '03-16', '04-14', '04-15', '04-16', '04-17', '04-18', '05-24', '05-25']

In [ ]:
dates = ['04-14', '04-15']

In [ ]:
se_no = 1
for dat in dates:
    print(dat)
    reqout = requests.get(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')

    for fil in range(2, 100):

        params = {
            'coday': f'2023-{dat}',
            'embed': '1',
            'advSearchThemeList': f'{fil}' }

        reqout = requests.get('https://eccmid2023.key4.live/programme-live-1', params=params, headers=headers)
        soup = BS(reqout.content, 'lxml')

        outer_row = soup.find_all(class_='row live-program-card-row-grid')
        locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')

        for ou in outer_row:

            col_div = 0

            for i_id in ou.find_all(class_='border text-center program-grid-calandar'):

                if i_id.find(class_='programme-session-card programme-session-display border'):

                    session_number = f'S{se_no}'
                    print(f'=====**====={session_number}=====**=======', col_div)
                    i_id = i_id.find(class_='programme-session-card programme-session-display border')

                    data['sessionRef'] = i_id.get('data-session-reference')
                    
                    if len(str(soup)) < 20:
                        print('%%%%%%%%%%%%% check %%%%%%%%%%%%%')
                        print(i_id.get('data-session-reference'))
                        continue
                        
                    req = requests.post(
                        'https://eccmid2023.key4.live/fo-user-display-session-details.php',
                        cookies=cookies,
                        headers=headers,
                        data=data,)

                    soupin = BS(req.content, 'lxml')
                    sess_data = soupin.find(id='modal-session-content')

                    # -------------------------------------    date time location ya type     ------------------------------------
                    sess_date = ''
                    sess_time = ''
                    sess_loc = ''
                    sess_type = ''
                    sess_loc = ''
                    sess_loc = locations[col_div].text.strip()
                    date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                    sess_date = date_time_type.split(',')[0].strip()
                    sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                    sess_type = date_time_type.split('CET')[-1].strip()

                    # --------------------------------------  category or type and title -----------------------------------------------------
                    sess_cate = ''
                    sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                    sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                    sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                    # --------------------------------------- abstract text OR co organised ---------------------------------------------
                    co_org = ''
                    sess_text = ''
                    if sess_data.find(class_='modal-session-organized-by mt-2'):
                        co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                    if sess_data.find(class_='modal-session-desc mt-2'):
                        sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()

                    sess_text = co_org + ' ' + sess_text

                    # ------------------------------------------- session ka authors aff --------------------------------------------------
                    sess_author = ''
                    sess_aff = ''

                    if sess_data.find(class_='modal-session-moderators'):
                        for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                            aff = i.find(class_='modal-session-moderator-country').text.strip()
                            auth = i.text.strip()
                            auth = auth.replace(aff, '').split('-')[0]
                            aff = aff.replace(',', '').strip()

                            sess_author += auth + '; '
                            sess_aff += aff + '; '

                    sess_author = clean_semicolon(sess_author)
                    sess_aff = clean_semicolon(sess_aff)
                    sess_date = sess_date + '/2023'
                    sess_type = re.sub(sess_loc, '', sess_type)
                    
                    
                    art_sponser = ''
                    if soupin.find(class_='modal-session-organized-by mt-2'):
                        spons = soupin.find(class_='modal-session-organized-by mt-2').text.strip()
                        if re.search('^Organized ', spons, flags=re.S):
                            art_sponser = spons
                    
                    print(f'sess date :- {sess_date}')
                    print(f'sess time :- {sess_time}')
                    print(f'sess loca :- {sess_loc}')
                    print(f'sess type :- {sess_type}')
                    print(f'sess cate :- {sess_cate}')
                    print(f'sess title :- {sess_title}')
                    print(f'abs text :- {sess_text}')
                    print(f'sess auth :- {sess_author}')
                    print(f'sess aff :- {sess_aff}')

                    dic['session_id'].append(session_number)
                    dic['news_type'].append('Session')
                    dic['source_id'].append('')
                    dic['manual_id'].append('')
                    dic['article_title'].append(sess_title)
                    dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                    dic['authors'].append(sess_author)
                    dic['author_affiliation'].append(sess_aff)
                    dic['abstract_text'].append(sess_text)
                    dic['date'].append(sess_date)
                    dic['start_time'].append(sess_time.split('-')[0].strip())
                    dic['end_time'].append(sess_time.split('-')[-1].strip())
                    dic['location'].append(sess_loc)
                    dic['session_title'].append(sess_title)
                    dic['session_type'].append(sess_type)
                    dic['category'].append(sess_cate)
                    dic['sub_category'].append('')
                    dic['disclosure'].append('')
                    print('======'*20)

                    # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*

                    abs_data = sess_data.find(class_='modal-sessions-interventions')
                    if abs_data:

                        pno = 1
                        for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                            abs_title = ab.find(style='font-weight: bold').text.strip()

                            # abs author
                            abs_author = ''
                            abs_aff = ''
                            if ab.find(class_=re.compile('modal-session-faculties mb-1')):
                                
                                ab_authaff = ab.find(class_=re.compile('modal-session-faculties mb-1')).text.strip()
                                abs_author = ', '.join(ab_authaff.split(',')[:-1])
                                abs_aff = ab_authaff.split(',')[-1]

                            if abs_title == '':
                                abs_title = 'TBD'

                            # source id...... time shit
                            
                            abs_text = ''
                            ar_id = ''
                            art_time = ''

                            if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                                src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                                if src_code:

                                    if re.search('^\d{1,6}', src_code):
                                        ar_id = src_code + '-' + str(pno)
                                        art_time = ''
                                    elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                        str_pno = str(pno)
                                        if len(str_pno) == 1:
                                            str_pno = '0' + str_pno
                                        ar_id = src_code + str_pno
                                        art_time = sess_time
                            else:
                                ar_id = 'QC'
                                
                            abs_text = art_sponser + ' ' + abs_text
                            print(ar_id, '===', art_time)
                            print(f'abs title :- {abs_title}')
                            print(f'abs auth :- {abs_author}')
                            print(f'abs aff :- {abs_aff}')

                            dic['session_id'].append(session_number)
                            dic['news_type'].append('Abstract')
                            dic['source_id'].append(ar_id)
                            dic['manual_id'].append('')
                            dic['article_title'].append(abs_title)
                            dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                            dic['authors'].append(abs_author)
                            dic['author_affiliation'].append(abs_aff)
                            dic['abstract_text'].append('')
                            dic['date'].append(sess_date)
                            dic['start_time'].append(sess_time.split('-')[0].strip())
                            dic['end_time'].append(sess_time.split('-')[-1].strip())
                            dic['location'].append(sess_loc)
                            dic['session_title'].append(sess_title)
                            dic['session_type'].append(sess_type)
                            dic['category'].append(sess_cate)
                            dic['sub_category'].append('')
                            dic['disclosure'].append('')
                            print('-*-+-'* 17)
                            pno += 1

                    se_no += 1
                    print('inr', se_no)

                col_div += 1
            

In [ ]:
df = pd.DataFrame(dic)
df

In [ ]:
df1 = pd.read_excel('ecc2.xlsx')
df2 = pd.read_excel('ecc3.xlsx')

In [ ]:
# res = pd.concat([df, df1, df2])
res

In [ ]:
res.to_excel('eccmid_FilterAll_noPaid.xlsx', index=False)

In [ ]:
dfal = pd.read_excel("eccmid_kkk.xlsx")
dfal

In [ ]:
naya = pd.concat([dfal, res])
naya

In [ ]:
naya.to_excel("eccmid_last.xlsx", index=False)

### giving proper session id if two excel used for scraping one link

In [ ]:
dfpaid = pd.read_excel('eccmid_YesPaid.xlsx')
dfno = pd.read_excel('eccmid_noPaid.xlsx')

In [ ]:
c = 0
sno = 0
for i in res['session_title']:
    
    stp = res.iloc[c]['news_type']
    if stp == 'Session':
        sno += 1
        
    sesid = f'S{sno}'
    print(i, stp, sesid, sno)
    res.iat[c, res.columns.get_loc('session_id')] = sesid
    
    if str(i) == 'nan':
        break
    c += 1

In [ ]:
dfpaid.to_excel('eccmid_paidD.xlsx', index=False)

In [ ]:
dfno.to_excel('eccmimd_noPaidD.xlsx', index=False)

In [ ]:
df.to_excel('posterswala_eccmid_time.xlsx', index=False)

In [ ]:
req = requests.get('https://eccmid2023.key4.live/programme-live-1?coday=2023-04-15&embed=1')
soup = BS(req.content, 'lxml')

In [ ]:
se = set()
for i in list(soup.find_all(class_='p-3 program_advanced-search_section')[2].find(class_='row').children):
    if len(str(i)) > 4:
        print(i.find('input').get('data-id'), i.find('input').get('value'))
        se.add(i.find('input').get('value'))

In [ ]:
for tit in se:
    c = 0
    for i in df['article_title']:
        if re.search(tit, i):
            print('found', c)
        c += 1

In [ ]:
import PyPDF2
def return_pdf_text(pdf_name):
    pdfFileObj = open(pdf_name, 'rb')

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    abs_text = ''
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        abs_text += pageObj.extractText() + ' '
    
    pdfFileObj.close()
    return abs_text

In [ ]:
pdf_text = return_pdf_text('anuj_pdf.pdf')

In [ ]:
sara_codes = set()

c = 0
for i in df['session_title']:
    
    if df.iloc[c]['session_title'] == df.iloc[c]['article_title']:
        sara_codes.add(i.split('-')[0].strip())
        
    c += 1

In [ ]:
len(sara_codes)

In [ ]:
len(sara_codes)

In [ ]:
sara_code_pdf = set()

c = 0
for i in sara_codes:
    if re.search(i, pdf_text, flags=re.S) and re.search(f'{i} ', pdf_text, flags=re.S):
        sara_code_pdf.add(i)
        c += 1
        
print(c)

In [ ]:
c = 0
co = 0
for i in df['article_title']:
    
    if df.iloc[c]['session_title'] != df.iloc[c]['article_title'] and df.iloc[c]['session_title'].split('-')[0].strip() in sara_code_pdf:
        
        if '(' in i:
            search_text = re.search('(.*)\(.*?\)(.*)', i).group(1)
            search_text = search_text + '.*' + re.search('(.*)\(.*?\)(.*)', i).group(2)
        else:
            search_text = i
            
            
            if len(re.findall(search_text, new_pdf_text, flags=re.S)) == 0:
                co += 1

            else:
                print(len(re.findall(search_text, new_pdf_text, flags=re.S)))


                stin = re.search(search_text, new_pdf_text, flags=re.S).span()[0] - 14
                enin = re.search(search_text, new_pdf_text, flags=re.S).span()[1]
                time_srcid = new_pdf_text[stin:enin]

                time_abs = ''
                srcid = ''
                if re.search('\d{1,2}:\d{1,2}', time_srcid):
                    time_abs = re.search('\d{1,2}:\d{1,2}', time_srcid).group()

                if re.search('\d{3,4}|\d{3,4}-\d{1}', time_srcid):
                    srcid = re.search('\d{3,4}-\d{1}|\d{3,4}', time_srcid).group()

                print(time_abs, '==========', srcid, '--', df.iloc[c]['session_title'].split('-')[0].strip())
                if time_abs == '' or srcid == '':
                    print('checkkkk =============')
    
#                 df.iat[c, df.columns.get_loc('start_time')] = time_abs
#                 df.iat[c, df.columns.get_loc('source_id')] = srcid
    c += 1

In [ ]:
df.to_excel('pdf_edited_eccmid.xlsx', index=False)